### Styles result CropType .geojson. Note, that you have to add original path to results .geojson file obtained after learning model. See on pw folder and additional notebooks on server.

In [6]:
import os
from typing import Dict, List

import geopandas as gpd
import numpy as np
import pandas as pd
import pyproj
import rasterio
import torch
import json
from rasterio.transform import from_origin, rowcol
from rasterio.windows import Window
from shapely.geometry import Polygon
from torch.nn import functional as F
from tqdm import tqdm
from pathlib import Path

from modelling.raster_predict import predict

In [7]:
# use single GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [8]:
BASE = f"/home/{os.getenv('NB_USER')}/work"

WORKDIR = os.path.join(BASE, "notebooks/pw")
RESULTSDIR = os.path.join(BASE, "results/pw")

In [9]:
class Args:
    def __init__(self):
        self.batch_size = 1
        self.model_weights_path = os.path.join(WORKDIR, "data/experiments/mi_clstm_192_mask_64_crop_s1_s2_5_months_10m/logs/checkpoints/best.pth")
        self.experiments_dir = os.path.join(WORKDIR, "data/experiments/")
        self.experiment_name = "mi_clstm_192_mask_64_crop_s1_s2_5_months_10m"
        self.mask_size = 192
        self.crop_size = 64
        self.num_classes = 7
        self.model_name = 'mi_clstm'
        self.satellites = ["s1", "s2"]
        # Args for CLSTM model
        self.hidden_dims = 128
        self.crnn_kernel_sizes = 3
        self.conv_kernel_size = 3
        self.crnn_num_layers = 1
        self.bidirectional = False
        self.avg_hidden_states = True
        self.d_attn_dim = 32
        self.r_attn_dim = 1
        self.dk_attn_dim = 256
        self.dv_attn_dim = 256
        self.num_timesteps = 40
        self.segm_model = "baseline"
        self.months_to_include = ["4", "5", "6", "7", "8"]

        # Raster prediction args
        self.class_names_json = os.path.join(WORKDIR, "data/labels/ohio/cropscape_labels/merged_classes/final_classes.json")
        self.clu_info_path = os.path.join(WORKDIR, "data/labels/ohio/provided_data/ohio_CLUs_2020.geojson")
        self.raster_resolution = 10
        self.sentinel1_dir = os.path.join(WORKDIR, "data/sentinel1/")
        self.sentinel2_dir = os.path.join(WORKDIR, "data/sentinel2/")
        self.year = 2020
        self.predicted_raster_path = os.path.join(WORKDIR, "data/final_predictions/predictions_test.tif")
        self.predicted_clus_path = os.path.join(WORKDIR, "data/final_predictions/clu_predictions_test.geojson")
        self.aoi_path = os.path.join(WORKDIR, "data/labels/ohio/ohio_AOI.geojson")
        self.main_attn_type = 'None'
args = Args()

In [ ]:
predict(args)

In [10]:
predictions_path = os.getenv("PREDICTIONS", args.predicted_clus_path) 

if not predictions_path:
    raise RuntimeError("Add PREDICTIONS env var for calculations") 
predictions_path

'/home/jovyan/work/notebooks/pw/data/final_predictions/clu_predictions_test.geojson'

In [13]:
def read_geojson(path, crs="epsg:4326"):
    df = gpd.read_file(predictions_path)
    if str(df.crs) != crs:
        df.to_crs(crs, inplace=True)
    return df

In [14]:
preds = read_geojson(predictions_path)[["geometry", "class_name"]]
preds

,geometry,class_name
0,"MULTIPOLYGON (((-80.92192 40.63885, -80.92126 ...",Other
1,"MULTIPOLYGON (((-80.93056 40.69504, -80.92921 ...",Corn
2,"MULTIPOLYGON (((-80.89612 40.63810, -80.89616 ...",Soybeans
3,"MULTIPOLYGON (((-80.93498 40.69577, -80.93520 ...",Corn
4,"MULTIPOLYGON (((-80.89639 40.64179, -80.89650 ...",Other
...,...,...
243,"MULTIPOLYGON (((-83.38668 39.68851, -83.38131 ...",Corn
244,"MULTIPOLYGON (((-83.27544 39.57453, -83.27367 ...",Other
245,"MULTIPOLYGON (((-83.38905 39.68240, -83.38817 ...",Other
246,"MULTIPOLYGON (((-83.25731 39.50507, -83.25765 ...",Corn


In [15]:
def create_style(class_):
    colors = dict(soybeans='#d6bb87',
                  corn='#fbec5d')
    
    style = dict(color=colors.get(class_.lower(), '#C0C0C0'), 
                 popup_text=class_.lower(), 
                )    
    return str(style)

In [16]:
preds['style'] = preds['class_name'].apply(lambda cl: create_style(cl))
preds

,geometry,class_name,style
0,"MULTIPOLYGON (((-80.92192 40.63885, -80.92126 ...",Other,"{'color': '#C0C0C0', 'popup_text': 'other'}"
1,"MULTIPOLYGON (((-80.93056 40.69504, -80.92921 ...",Corn,"{'color': '#fbec5d', 'popup_text': 'corn'}"
2,"MULTIPOLYGON (((-80.89612 40.63810, -80.89616 ...",Soybeans,"{'color': '#6F634B', 'popup_text': 'soybeans'}"
3,"MULTIPOLYGON (((-80.93498 40.69577, -80.93520 ...",Corn,"{'color': '#fbec5d', 'popup_text': 'corn'}"
4,"MULTIPOLYGON (((-80.89639 40.64179, -80.89650 ...",Other,"{'color': '#C0C0C0', 'popup_text': 'other'}"
...,...,...,...
243,"MULTIPOLYGON (((-83.38668 39.68851, -83.38131 ...",Corn,"{'color': '#fbec5d', 'popup_text': 'corn'}"
244,"MULTIPOLYGON (((-83.27544 39.57453, -83.27367 ...",Other,"{'color': '#C0C0C0', 'popup_text': 'other'}"
245,"MULTIPOLYGON (((-83.38905 39.68240, -83.38817 ...",Other,"{'color': '#C0C0C0', 'popup_text': 'other'}"
246,"MULTIPOLYGON (((-83.25731 39.50507, -83.25765 ...",Corn,"{'color': '#fbec5d', 'popup_text': 'corn'}"


In [ ]:
print("Generating styled result .geojson file...")

stem = Path(predictions_path).stem
temp_path = os.path.join(BASE, f"{RESULTSDIR}/{stem}.geojson.temp") 
os.makedirs(os.path.dirname(temp_path), exist_ok=True)

preds.to_file(temp_path, driver='GeoJSON')

print(f"Generated temp file: {temp_path}")
output_path = temp_path[:-5]
print(f"Rename temp file: {temp_path} to {output_path}")
os.rename(temp_path, output_path)

print("Execution finished")